Loading the datasets and checking their shape

In [2]:
    import pandas as pd

    # Load the datasets
    sales = pd.read_csv('/Users/ngachoe2002/Desktop/inventoryai/data/sales_train_validation.csv')
    calendar = pd.read_csv('/Users/ngachoe2002/Desktop/inventoryai/data/calendar.csv')
    sell_prices = pd.read_csv('/Users/ngachoe2002/Desktop/inventoryai/data/sell_prices.csv')

    # Show the shapes and first few rows of each dataset
    print("Sales shape:", sales.shape)
    display(sales.head())

    print("Calendar shape:", calendar.shape)
    display(calendar.head())

    print("Sell prices shape:", sell_prices.shape)
    display(sell_prices.head())


Sales shape: (30490, 1919)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


Calendar shape: (1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


Sell prices shape: (6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


Preparing and executing wide-to-long reshape

In [3]:
# --- 1. Prepare Calendar Data for Merging ---
# Select only the minimal calendar columns needed for date mapping and basic features.
calendar_cols = ['d', 'date', 'wm_yr_wk', 'event_name_1'] 
calendar_slim = calendar[calendar_cols].copy()
calendar_slim = calendar_slim.rename(columns={'date': 'ds'})

# --- 2. Define Melt Variables for Full Sales Data ---
# These are the descriptive columns that stay as IDs (no change from original)
id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# These are the columns to be collapsed—all the daily sales columns (d_1, d_2, ...)
value_vars = [col for col in sales.columns if col.startswith('d_')]

print("Melt variables defined successfully.")

Melt variables defined successfully.


In [4]:
print("Starting Wide-to-Long Reshape...")

sales_melted = sales.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='d',           # New column: Day ID (e.g., d_1)
    value_name='y'          # New column: Sales count (Prophet target)
)

print("Reshape Complete.")
print(f"Melted Sales shape: {sales_melted.shape}")
display(sales_melted.head())

Starting Wide-to-Long Reshape...
Reshape Complete.
Melted Sales shape: (58327370, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,y
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


Map dates,clean and add features

In [5]:
# --- 1. Merge Day ID to Actual Date ('ds') ---
# Merge the 58 million sales rows with the calendar data on the common 'd' (Day ID) column.
df_final_full = pd.merge(sales_melted, calendar_slim, on='d', how='left')

# --- 2. Convert Date and Drop Temporary Columns ---
# Convert 'ds' to the required datetime format
df_final_full['ds'] = pd.to_datetime(df_final_full['ds'])

# Drop the generic 'd' column, which is no longer needed
df_final_full.drop(columns=['d'], inplace=True)

# --- 3. Feature Engineering: Create Simple Event Flag ---
# Check if an event occurred on a given day (for use as a regressor in Prophet)
df_final_full['is_event'] = df_final_full['event_name_1'].apply(lambda x: 1 if pd.notna(x) else 0)

# Drop the original event name column after creating the flag
df_final_full.drop(columns=['event_name_1'], inplace=True)

print(f"\nFinal Merged Data Shape: {df_final_full.shape}")
print(f"Columns ready for modeling: {df_final_full.columns.tolist()}")
display(df_final_full[['ds', 'y', 'store_id', 'is_event']].head())


Final Merged Data Shape: (58327370, 10)
Columns ready for modeling: ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'y', 'ds', 'wm_yr_wk', 'is_event']


,ds,y,store_id,is_event
0,2011-01-29,0,CA_1,0
1,2011-01-29,0,CA_1,0
2,2011-01-29,0,CA_1,0
3,2011-01-29,0,CA_1,0
4,2011-01-29,0,CA_1,0


Saving the processed csv as parquet file 

In [6]:
# --- SAVE DATA ---
# Define the output path pointing to the efficient Parquet file format
OUTPUT_PATH = '../data/processed/df_final_full.parquet'

# Note: You must manually create the 'processed' directory inside your 'data' folder first.
# to_parquet is faster and maintains memory-efficient data types.
df_final_full.to_parquet(OUTPUT_PATH, index=False)